In [ ]:
#https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72
#https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-coding-edd8f1cf8f2d

#https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics

In [3]:
from sklearn import svm

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneOut

In [13]:
iris_aux = datasets.load_iris()
iris = pd.DataFrame(iris_aux.data, columns=iris_aux.feature_names)
iris['target'] = pd.Series(iris_aux.target)
iris['species'] = pd.Categorical.from_codes(iris_aux.target, iris_aux.target_names)
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [15]:
X = iris.iloc[:,:4]
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [17]:
y = iris.target
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int32

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [21]:
cv_kfold = KFold(n_splits=3, shuffle=False, random_state=None)

In [22]:
clf = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, 
              shrinking=True, probability=False, tol=0.001, cache_size=200, 
              class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)
#svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, 
        #shrinking=True, probability=False, tol=0.001, cache_size=200, 
    #class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

parameters = {
    #'penalty' : ['l2'],
    #'fit_intercept': [True] ,
    #'solver': ['liblinear']
}

#scoring pode ser: accuracy, roc_auc, recall, precision... 
####
#Classification	 	 
#‘accuracy’	metrics.accuracy_score	 
#‘average_precision’	metrics.average_precision_score	 
#‘f1’	metrics.f1_score	for binary targets
#‘f1_micro’	metrics.f1_score	micro-averaged
#‘f1_macro’	metrics.f1_score	macro-averaged
#‘f1_weighted’	metrics.f1_score	weighted average
#‘f1_samples’	metrics.f1_score	by multilabel sample
#‘neg_log_loss’	metrics.log_loss	requires predict_proba support
#‘precision’ etc.	metrics.precision_score	suffixes apply as with ‘f1’
#‘recall’ etc.	metrics.recall_score	suffixes apply as with ‘f1’
#‘roc_auc’	metrics.roc_auc_score

model_svm = GridSearchCV(estimator = clf, param_grid = parameters, scoring= 'accuracy', fit_params=None, n_jobs=1, 
                         iid=True, refit=True, 
                     cv=cv_kfold, verbose=0, pre_dispatch='2*n_jobs', error_score='raise', return_train_score= True)
model_svm

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [23]:
grid_svm = model_svm.fit(X_train, y_train)

In [24]:
pd.DataFrame(grid_svm.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.058657,0.082953,0.0,0.0,{},0.921053,1.0,0.918919,0.946429,0.037637,1,0.972973,0.973333,1.0,0.982102,0.012657


In [25]:
model_final_svm = grid_svm.best_estimator_

In [27]:
predict_svm = model_final_svm.predict(X_test)

In [31]:
metrics.accuracy_score(y_test, predict_svm)

0.9736842105263158

In [32]:
pd.DataFrame(metrics.confusion_matrix(y_test, predict_svm))

,0,1,2
0,13,0,0
1,0,15,1
2,0,0,9
